In [2]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import models.bioembeddings_dallago.model_utils as model_utils

/projects/ashehu/akabir4/venvs/hopper_bioembeddings_dallago_from_source/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model, tokenizer, model_name = model_utils.get_model_tokenizer("prottrans_t5_xl_u50") # prottrans_t5_xl_u50, prottrans_bert_bfd, plus_rnn


Log: Model loading ...


In [4]:
model._model

T5ForConditionalGeneration(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dro

In [15]:
print(tokenizer)
print(tokenizer.mask_token)
print(tokenizer.eos_token)
print(tokenizer.unk_token)

print(tokenizer.convert_ids_to_tokens(2), tokenizer.convert_tokens_to_ids('[CLS]')) # <mask> is unknown here
print(tokenizer.convert_ids_to_tokens(3), tokenizer.convert_tokens_to_ids('[SEP]')) # <mask> is unknown here
print(tokenizer.convert_ids_to_tokens(28), tokenizer.convert_tokens_to_ids('<extra_id_99>')) # <mask> is unknown here
print(tokenizer.convert_ids_to_tokens(6), tokenizer.convert_tokens_to_ids('▁V'))

Using mask_token, but it is not set yet.


T5Tokenizer(name_or_path='/home/akabir4/.cache/bio_embeddings/prottrans_t5_xl_u50/model_directory', vocab_size=128, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_

In [16]:
import re
seq = "ASDX"

seq = re.sub(r"[UZOB]", "X", seq) # replacing unknown amino acid with unknown token
seq = list(seq)

mut_pos_zero_idxed = 2 # the outputs of mutant A_DX and AS_X are different at every positions.
seq[mut_pos_zero_idxed] = '<extra_id_99>'# tokenizer.mask_token #'<extra_id_0>' # mut_pos must be 0-indexed. replace AA by special mask token used by the model

seq = " ".join(list(seq)) # space separated amino acids
print(seq)

A S <extra_id_99> X


In [17]:
# <cls> token at the end
# starts from 0-index
input_ids = tokenizer.batch_encode_plus(
            [seq], add_special_tokens=True, padding="longest"
        )
print(input_ids)

import torch
print(model._device)
tokenized_sequences = torch.tensor(input_ids["input_ids"]).to(model._device)
attention_mask = torch.tensor(input_ids["attention_mask"]).to(model._device)

{'input_ids': [[3, 7, 28, 23, 1]], 'attention_mask': [[1, 1, 1, 1, 1]]}
cuda


In [18]:
with torch.no_grad():
    embeddings = model._model(input_ids=tokenized_sequences, attention_mask=attention_mask, decoder_input_ids=tokenized_sequences)
embeddings = embeddings[0].squeeze().cpu().numpy()
embeddings = embeddings[0:4]
print(embeddings)
print(embeddings.shape)

[[-14.350241  -11.687776  -42.42467    -3.4729238  -5.964431   -4.8893976
   -6.125003   -4.8880596  -5.156353   -5.9897866  -6.4371357  -6.2315335
   -7.9434457  -5.1404295  -7.2725296  -7.605296   -6.6906786  -7.861474
   -8.368013   -7.968816   -7.130129   -7.6022234  -7.2519426  -2.7361221
  -42.645756  -42.813927  -42.70206   -42.29748   -41.80173   -42.87064
  -42.28718   -42.788235  -42.385338  -42.98429   -42.630558  -42.968315
  -42.468292  -42.322624  -42.810974  -42.7588    -41.675877  -42.703617
  -42.893196  -42.880375  -41.771393  -42.115273  -42.540512  -41.34042
  -42.190464  -41.98073   -42.344368  -42.82348   -42.247337  -41.713467
  -42.780758  -42.40633   -41.786613  -41.908684  -42.7054    -42.3744
  -42.590557  -41.829338  -42.69959   -42.343628  -41.48479   -42.54636
  -41.30416   -41.50154   -42.04083   -43.01085   -42.833954  -42.825424
  -43.050713  -42.636925  -42.135227  -41.215317  -42.364098  -42.860554
  -42.322365  -42.762142  -42.397606  -42.55921   -42

In [19]:
logits = model.get_masked_logits("ASDX", 2)
logits

array([[-17.390213 ,  -9.293229 , -50.62278  ,  -7.236208 , -10.624226 ,
         -8.695352 , -10.065985 ,  -9.033566 ,  -9.4142275,  -9.940994 ,
        -10.468841 , -10.259579 , -12.276775 ,  -9.225727 , -11.641764 ,
        -12.02408  , -11.323843 , -12.420125 , -12.841397 , -12.613511 ,
        -11.722965 , -12.178923 , -11.511996 ,  -4.7033224, -50.69864  ,
        -51.003494 , -50.929955 , -50.378647 , -49.787434 , -51.267982 ,
        -50.5732   , -51.000885 , -50.635746 , -51.379807 , -50.90525  ,
        -51.337032 , -50.69402  , -50.55966  , -51.067017 , -51.027008 ,
        -49.835648 , -50.958683 , -51.25965  , -51.352943 , -50.013577 ,
        -50.38726  , -50.79406  , -49.55082  , -50.35047  , -50.089252 ,
        -50.655922 , -51.105316 , -50.492157 , -49.899284 , -51.121902 ,
        -50.767487 , -49.833534 , -49.9345   , -50.97247  , -50.68561  ,
        -51.007282 , -50.040413 , -51.130936 , -50.581726 , -49.52233  ,
        -50.70136  , -49.259468 , -49.574875 , -50.